# MLP Implementation in mlax with Optax optimizers.
This notebook uses the [Optax](https://optax.readthedocs.io/en/latest/optax-101.html) JAX optimization library.

You can view the Pytorch reference implementation in `mlp_reference.ipynb`.

In [1]:
import jax
import jax.numpy as jnp
from jax import (
    nn,
    random,
    tree_util
)
import numpy as np
import optax

We import `Linear`, `Bias`, and `F` transformations from `mlax.nn` to build some
dense layers.

We import `Series` from `mlax.block` to stack the dense layers into an MLP.

In [2]:
from mlax.nn import Linear, Bias, F
from mlax.block import Series 

We import helpers to load data from Pytorch.

In [3]:
from dataloader import batch, load_mnist

### Load in and batch the MNIST datasets.
We use helper functions to load in Pytorch datasets as numpy and convert them in
to lists containing the batches.

Checkout
[Training a Simple Neural Network with tensorflow/datasets Data Loading](https://jax.readthedocs.io/en/latest/notebooks/neural_network_with_tfds_data.html) and
[Training a Simple Neural Network, with PyTorch Data Loading](https://jax.readthedocs.io/en/latest/notebooks/Neural_Network_and_Data_Loading.html)
for other ways to load in Tensorflow and Pytorch datsets.

In [4]:
# Load in datasets with helper
(X_train, y_train), (X_test, y_test) = load_mnist("../data")
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# Batch datasets
batch_size = 128
X_train, y_train = batch(X_train, y_train, batch_size)
X_test, y_test = batch(X_test, y_test, batch_size)
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)
469 469
79 79


### Initialize MLP model parameters.
`model_init` consumes a `jax.random.PRNGKey` when initializing the parameters.
Read more about random numbers in JAX [here](https://jax.readthedocs.io/en/latest/jax-101/05-random-numbers.html).

`F` is a wrapper around a stateless function. `Linear` is a linear
transformation without bias. `Bias` adds a bias term.

Each `init` function returns a `trainables`, `non_trainables`, and
`hyperparams`.

`trainables` are trainable weights. `non_trainables` are non-trainable
variables. `hyperparams` are additional parameters required by the forward pass.

The `trainables` and `non_trainables` are PyTrees of JAX arrays. Read more about
JAX PyTrees [here](https://jax.readthedocs.io/en/latest/pytrees.html).

`hyperparams` is a NamedTuple containing Python types.

In [5]:
def model_init(key):
    keys_iter = iter(random.split(key, 6))
    return Series.init(
        # Convert int8 numpy inputs to float32 JAX arrays and flatten them
        F.init(lambda x:jnp.reshape(
            jnp.asarray(x, jnp.float32) / 256.0,
            (len(x), -1))
        ),

        # Dense layer with relu activation
        Linear.init(
            key=next(keys_iter),
            in_features=28 * 28,
            out_features=512,
        ),
        Bias.init(
            key=next(keys_iter),
            in_feature_shape=(512,)
        ),
        F.init(nn.relu),

        # Dense layer with relu activation
        Linear.init(next(keys_iter), 512, 512),
        Bias.init(next(keys_iter), (512,)),
        F.init(nn.relu),
        
        # Dense layer without activation or softmax
        Linear.init(next(keys_iter), 512, 10),
        Bias.init(next(keys_iter), (10,)),
    )

trainables, non_trainables, hyperparams = model_init(random.PRNGKey(0))

### Define MLP dataflow.
`Series.fwd` takes in batched input features and tuples of `trainables`,
`non_trainables`, and `hyperparams`. It figures out which layer each
`hyperparams` is for, and calls their forward pass functions on the input
features in sequence.

It returns the model predictions and updated `non_trainables`.

We jit-compile the forward pass function for significant speedups. Note
that `hyperparams` is a static argument because it is made of Python types, not
valid JAX types, and it also used interally for control flow. Read more about
jit-compile [here](https://jax.readthedocs.io/en/latest/jax-101/02-jitting.html).

In [6]:
@tree_util.Partial(jax.jit, static_argnames="hyperparams")
def model_fwd(
    x_batch: np.array,
    trainables,
    non_trainables,
    hyperparams
):
    return Series.fwd(x_batch, trainables, non_trainables, hyperparams)

### Define loss function.
The loss function is also jit-compiled for better performance.

In [7]:
@jax.jit
def loss_fn(
    batched_preds: jnp.array,
    batched_targets: np.array
):
    return optax.softmax_cross_entropy_with_integer_labels(
        batched_preds,
        batched_targets
    ).mean() # Optax returns per-example loss, this returns the mean batch loss

We define a convenience function that get model predictions on batched inputs,
and calculates the loss against batched targets.

It returns the batch loss and updated `non_trainables`.

In [8]:
def model_loss(
    x_batch: np.array,
    y_batch: np.array,
    trainables,
    non_trainables,
    hyperparams,
):
    preds, non_trainables = model_fwd(
        x_batch, trainables, non_trainables, hyperparams
    )
    return loss_fn(preds, y_batch), non_trainables

### Define optimizer using Optax.
We pass the `trainables` to `init` to initialize an optimizer state.

We define a function that takes in `trainables` gradients and an `optim_state`,
and returns updates to be applied on `trainables` and a new `optim_state`.

Note we used `jax.tree_util.Partial` to wrap the `optim_fn`. Doing so allows the
`optim_fn` to be passed to jit-compiled functions, notably `train_step`.
Read more about this [here](https://jax.readthedocs.io/en/latest/_autosummary/jax.tree_util.Partial.html?highlight=Partial).

In [9]:
optimizer = optax.sgd(1e-2, momentum=0.9)
optim_state = optimizer.init(trainables)
optim_fn = tree_util.Partial(optimizer.update)

### Define training step.
We  use JAX's `value_and_grad` to calculate the batch loss and
gradients with respect to the `trainables`. Read more about JAX's autodiff
[here](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html#taking-derivatives-with-grad).

The batch loss is only used for logging, but the gradients are passed to
`optim_fn` to get update gradients and a new `optim_state`.

We apply the update gradient on the model weights.

Finally, we return the batch loss, new `trainables`, and the new `optim_state`.

In [10]:
@tree_util.Partial(jax.jit, static_argnames="hyperparams")
def train_step(
    x_batch: np.array, 
    y_batch: np.array,
    trainables, non_trainables, hyperparams,
    optim_fn, optim_state
):
    # Find batch loss and gradients
    (loss, non_trainables), gradients = jax.value_and_grad(
        model_loss,
        argnums=2, # gradients wrt trainables (argument 2 of model_loss)
        has_aux=True # non_trainables is auxiliary data, loss is the true ouput
    )(x_batch, y_batch, trainables, non_trainables, hyperparams)

    # Get new gradients and optimizer state
    gradients, optim_state = optim_fn(gradients, optim_state)

    # Update model_weights with new gradients
    trainables = optax.apply_updates(gradients, trainables)
    return loss, trainables, non_trainables, optim_state

### Define functions for training and testing loops.

In [11]:
def train_epoch(
    X_train, y_train,
    trainables, non_trainables, hyperparams,
    optim_fn, optim_state
):
    num_batches = len(X_train)
    train_loss = 0.0
    for i in range(num_batches):
        x_batch, y_batch = X_train[i], y_train[i]
        loss, trainables, non_trainables, optim_state = train_step(
            x_batch, y_batch,
            trainables, non_trainables, hyperparams,
            optim_fn, optim_state
        )
        train_loss += loss

    print(f"Train loss: {train_loss / num_batches}") 
    return trainables, non_trainables, optim_state

In [12]:
def test(
    X_test, y_test,
    trainables, non_trainables, hyperparams
):
    num_batches = len(X_test)
    test_loss, accuracy = 0, 0.0
    for i in range(num_batches):
        x_batch, y_batch = X_test[i], y_test[i]
        preds, _ = model_fwd(
            x_batch, trainables, non_trainables, hyperparams
        )
        loss = loss_fn(preds, y_batch)
        test_loss += loss
        accuracy += (jnp.argmax(preds, axis=1) == y_batch).sum() / len(x_batch)
    
    print(f"Test loss: {test_loss / num_batches}, accuracy: {accuracy / num_batches}")

In [13]:
def train_loop(
    X_train, y_train,
    X_test, y_test,
    trainables, non_trainables, hyperparams,
    optim_fn, optim_state,
    epochs, test_every
):
    for i in range(epochs):
        epoch = i + 1
        print(f"Epoch {epoch}\n----------------")
        trainables, non_trainables, optim_state = train_epoch(
            X_train, y_train,
            trainables, non_trainables, hyperparams,
            optim_fn, optim_state
        )
        if (epoch % test_every == 0):
            test(X_test, y_test, trainables, non_trainables, hyperparams)
        print(f"----------------")
    
    return trainables, non_trainables, optim_state

### Train MLP on the MNIST dataset.

In [15]:
new_trainables, new_non_trainables, new_optim_state = train_loop(
    X_train, y_train,
    X_test, y_test,
    trainables, non_trainables, hyperparams,
    optim_fn, optim_state,
    30, 5
)

Epoch 1
----------------
Train loss: 0.45177510380744934
----------------
Epoch 2
----------------
Train loss: 0.20381902158260345
----------------
Epoch 3
----------------
Train loss: 0.14817491173744202
----------------
Epoch 4
----------------
Train loss: 0.11549077928066254
----------------
Epoch 5
----------------
Train loss: 0.09347715973854065
Test loss: 0.09885384142398834, accuracy: 0.9701344966888428
----------------
Epoch 6
----------------
Train loss: 0.07749040424823761
----------------
Epoch 7
----------------
Train loss: 0.06524626165628433
----------------
Epoch 8
----------------
Train loss: 0.05567239224910736
----------------
Epoch 9
----------------
Train loss: 0.04795210435986519
----------------
Epoch 10
----------------
Train loss: 0.041537802666425705
Test loss: 0.07356490194797516, accuracy: 0.9758702516555786
----------------
Epoch 11
----------------
Train loss: 0.03617854043841362
----------------
Epoch 12
----------------
Train loss: 0.03161139786243439
---